# Native Bayes

In [6]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.model_selection import KFold
import nltk
nltk.download('wordnet')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re

class NBModel:
    def __init__(self):
        self.theta_j_1=None
        self.prob=None
        self.num_feature=0
        self.theta_k = [None] * 8
        self.theta_j_k = [None] * 8
        self.class_prob = [None] * 8

    def preProcess(self,train,test):
        vectorizer = CountVectorizer(binary=True,max_features=5000)
        vectorizer.fit(train)
        trainVec = vectorizer.transform(train).toarray()
        testVec = vectorizer.transform(test).toarray()
        self.num_feature = len(vectorizer.get_feature_names())
        return trainVec, testVec
    def preProcess2(self,test):
        vectorizer = CountVectorizer(binary=True,max_features=5000)
        vectorizer.fit(test)
        testV = vectorizer.transform(test).toarray()
        self.num_feature = len(vectorizer.get_feature_names())
        return testV


    def fit(self,x,y,k):
        theta_1 = (y == k).sum() / float(y.shape[0])
        theta_0 = (y != k).sum() / float(y.shape[0])
        self.prob=np.log((theta_1/theta_0))
        self.theta_j_1 = np.zeros(self.num_feature)
        predict_label = np.zeros(x.shape[0])
        self.class_prob[k-1] = theta_1/theta_0
        self.theta_k[k-1] = theta_1

        p0Num = np.ones(x.shape[1])
        p1Num = np.ones(x.shape[1])

        for j in range(self.num_feature):
            p1Num[j]+=np.sum(x[y == k,j])

        # Laplace smoothing
        self.theta_j_1[:] = p1Num[:]/ float((y == k).sum()+2)
        self.theta_j_k[k-1] = self.theta_j_1[:]

    def predict(self,x):
        llh_k = [None] * 8
        llh=np.zeros(x.shape[0])
        predict=[None]*(x.shape[0])

        for k in range(0,8):
            llh[:] = np.sum(x[:, :] * np.log(self.theta_j_k[k][:]) + (1 - x[:, :]) * np.log(
                (1 - self.theta_j_k[k][:])),axis=1)
            llh[:]+=self.class_prob[k]
            llh_k[k] = llh[:]
            llh=np.zeros(x.shape[0])
        return np.argmax(llh_k, axis=0) + 1

def Accu_eval2(y, predict):
    count = 0
    for i in range(0, len(y)):
        if predict[i] ==y[i]:
            count+=1
    # print("The Accu is: ",accuracy)
    return count/len(y)

data = pd.read_csv('train.csv',dtype=str)
test = pd.read_csv('test.csv',dtype=str)
data = data.values
test = test.values
np.random.shuffle(data)
body = data[:, 0]
subReddit = data[:, 1]
testbody=test[:,1]
testIndex=test[:,0]

def check(word):
    word= word.lower()
    if bool(re.search(r'\d', word)) or bool(re.match(r'[^\w]', word)):
        return False
    else:
        return True

wnl = WordNetLemmatizer()
lemmatized = [None]*body.shape[0]
test_lemmatized = [None]*testbody.shape[0]
i = 0
for s in testbody:
    test_lemmatized[i] = " ".join([wnl.lemmatize(word) for word in word_tokenize(s) if (check(word))])
    i+=1
test_lemmatized=np.array(test_lemmatized)
i=0
for s in body:
    lemmatized[i] = " ".join([wnl.lemmatize(word) for word in word_tokenize(s) if (check(word))])
    i+=1
def result(re,fo,k):
    for i in range(1,len(fo)):
        if(fo[i]==1):
            re[i]=k
    return re
lemmatized=np.array(lemmatized)
print("lemmatized Done!!!!")

nb=NBModel()

#k fold
kf = KFold(n_splits=6, shuffle=False)
def toint(x):
    i=0
    if x=='rpg':
        i=1
    if x=='anime':
        i=2
    if x=='datascience':
        i=3
    if x=="hardware":
        i=4
    if x=="cars":
        i=5
    if x=="gamernews":
        i=6
    if x=="gamedev":
        i=7
    if x=='computers':
        i=8
    return i
def toString(x):
    if x == 1:
        return 'rpg'
    elif x == 2:
        return 'anime'
    elif x == 3:
        return 'datascience'
    elif x == 4:
        return "hardware"
    elif x == 5:
        return "cars"
    elif x == 6:
        return "gamernews"
    elif x == 7:
        return "gamedev"
    elif x == 8:
        return 'computers'
    else:
        return 'None'
t=data

result = [None]*testIndex.shape[0]
for c in range(0,len(t[:,1])):
    t[c,1]=toint(t[c,1])

temp=t[:,1]
AvgAccuV=0
AvgAccuT=0
print("In different K(6) fold:")
for train_index, test_index in kf.split(lemmatized):
    trainx = lemmatized[train_index]
    trainy =temp[train_index]
    valix = lemmatized[test_index]
    valiy = temp[test_index]
    train, vali = nb.preProcess(trainx, valix)
    for k in range(1,9):
        nb.fit(train,trainy,k)
    predictV = nb.predict(vali)
    predictT = nb.predict(train)
    AvgAccuV +=Accu_eval2(valiy,predictV)
    AvgAccuT += Accu_eval2(trainy,predictT)
    print("(Train set)The Accuracy for  is ", Accu_eval2(trainy,predictT))
    print("(Validation set)The Accuracy for  is ", Accu_eval2(valiy,predictV))


print("The Average of 6 fold")
print("(Train set)The Accuracy for  is ", AvgAccuT/6)
print("(Validation set)The Accuracy for  is ", AvgAccuV/6,"\n")


[nltk_data] Downloading package wordnet to /Users/guzeye/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/guzeye/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


lemmatized Done!!!!
In different K(6) fold:
(Train set)The Accuracy for  is  0.8345249196974407
(Validation set)The Accuracy for  is  0.7757638529259451
(Train set)The Accuracy for  is  0.8347321521085898
(Validation set)The Accuracy for  is  0.7897462454686691
(Train set)The Accuracy for  is  0.8394115209283051
(Validation set)The Accuracy for  is  0.7917098445595855
(Train set)The Accuracy for  is  0.8319519270617488
(Validation set)The Accuracy for  is  0.794818652849741
(Train set)The Accuracy for  is  0.8360961458765023
(Validation set)The Accuracy for  is  0.778238341968912
(Train set)The Accuracy for  is  0.8403439701616245
(Validation set)The Accuracy for  is  0.7756476683937824
The Average of 6 fold
(Train set)The Accuracy for  is  0.8361767726390351
(Validation set)The Accuracy for  is  0.7843207676944391 



# Test with different classifier with kfold

In [7]:
import numpy as np
import pandas as pd
import nltk
#nltk.download("popular")
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
import gc
import re

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectPercentile, f_classif, SelectFromModel
from sklearn.linear_model import SGDClassifier
data = pd.read_csv('train.csv',dtype=str)
test = pd.read_csv('test.csv',dtype=str)
data = data.values
test = test.values
np.random.shuffle(data)
body = data[:, 0]
subReddit = data[:, 1]
testbody=test[:,1]
testIndex=test[:,0]

def check(word):
    word= word.lower()
    if bool(re.search(r'\d', word)) or bool(re.match(r'[^\w]', word)):
        return False
    else:
        return True

wnl = WordNetLemmatizer()
lemmatized = [None]*body.shape[0]
test_lemmatized = [None]*testbody.shape[0]
i = 0
for s in testbody:
    test_lemmatized[i] = " ".join([wnl.lemmatize(word) for word in word_tokenize(s) if (check(word))])
    i+=1
test_lemmatized=np.array(test_lemmatized)
i=0
for s in body:
    lemmatized[i] = " ".join([wnl.lemmatize(word) for word in word_tokenize(s) if (check(word))])
    i+=1
lemmatized=np.array(lemmatized)

#ngram_range=(1, 2) "I love it" -> 'I' 'love' 'it' && 'I love' 'love it' 
#max_df ignore the words that the document frequency higher then x when building the voab list
#sublinear_tf use 1+log(tf) instead of tf


text_classiferLR =Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2),max_df=0.9, sublinear_tf=True)),
                      ('feature_selection', SelectPercentile(f_classif, percentile=40)),
                      ('Logistic', LogisticRegression(max_iter=2000))])

text_classiferDecisionTree = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2),max_df=0.9, sublinear_tf=True)),
                      ('feature_selection', SelectPercentile(f_classif, percentile=40)),
                      ('DT', DecisionTreeClassifier())])

text_classiferSGD =Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2),max_df=0.9, sublinear_tf=True)),
                      ('feature_selection', SelectPercentile(f_classif, percentile=40)),
                      ('SGD', SGDClassifier())])
# k fold with default parameter
kfCValidator = KFold(n_splits=5, shuffle=True,random_state=None)
def predictFactory(trainx,trainy,validationx,validationy,classifer,totalAccuaryV,totalAccuaryT):
    totalAccuaryV =0
    totalAccuaryT =0
    #fit
    classifer.fit(trainx,trainy)
    #predict
    predictV = classifer.predict(validationx)
    predictT = classifer.predict(trainx)
    #sum the accuray for each fold
    totalAccuaryV += np.mean(predictV == validationy)
    totalAccuaryT += np.mean(predictT == trainy)
    
    return totalAccuaryV, totalAccuaryT
    
    

totalAccuaryLRVali =0
totalAccuaryLRTrain =0    
totalAccuaryDecisionTreeVali =0
totalAccuaryDecisionTreeTrain =0 
totalAccuarySGDVali =0
totalAccuarySGDTrain =0 
for train_yield, test_yield in kfCValidator.split(lemmatized):
    trainx = lemmatized[train_yield]
    trainy = subReddit[train_yield]
    validationx = lemmatized[test_yield]
    validationy = subReddit[test_yield]
    
    addLRV, addLRT = predictFactory(trainx,trainy,validationx,validationy,text_classiferLR,totalAccuaryLRVali,totalAccuaryLRTrain);
    addDTV, addDTT = predictFactory(trainx,trainy,validationx,validationy,text_classiferDecisionTree,totalAccuaryLRVali,totalAccuaryLRTrain);
    addSGDV,addSGDT= predictFactory(trainx,trainy,validationx,validationy,text_classiferSGD,totalAccuaryLRVali,totalAccuaryLRTrain);
    totalAccuaryLRVali += addLRV
    totalAccuaryLRTrain += addLRT
    totalAccuaryDecisionTreeVali +=addDTV
    totalAccuaryDecisionTreeTrain +=addDTT
    totalAccuarySGDVali +=addSGDV
    totalAccuarySGDTrain+=addSGDT


LR_validation = totalAccuaryLRVali / 5.0
LR_train = totalAccuaryLRTrain / 5.0

DC_validation = totalAccuaryDecisionTreeVali/5.0
DC_train = totalAccuaryDecisionTreeTrain/5.0

SGD_validation = totalAccuarySGDVali/5.0
SGD_Train = totalAccuarySGDTrain/5.0
print("LogisticRegression Overall vali Accu:",LR_validation )
print("LogisticRegression Overall train Accu", LR_train)

print("DecisionTree Overall vali Accu:",DC_validation )
print("DecisionTree Overall train Accu", DC_train)


print("SGD Overall vali Accu:",SGD_validation )
print("SGD Overall train Accu", SGD_Train)


LogisticRegression Overall vali Accu: 0.8364712498965743
LogisticRegression Overall train Accu 0.9553401866452556
DecisionTree Overall vali Accu: 0.6581783438920705
DecisionTree Overall train Accu 0.9998273163882511
SGD Overall vali Accu: 0.8868074299519284
SGD Overall train Accu 0.9975393004664326


# Generate csv file for submission, tested by using test.csv

In [ ]:
import numpy as np
import pandas as pd
import nltk
#nltk.download("popular")
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
import gc
import re

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectPercentile, f_classif, SelectFromModel
from sklearn.linear_model import SGDClassifier
data = pd.read_csv('train.csv',dtype=str)
test = pd.read_csv('test.csv',dtype=str)
data = np.array(data)
test = np.array(test)

np.random.shuffle(data)
# body = data[:, 0]

body = (data[:, 0]).flatten()
subReddit = data[:, 1]
testbody = test[:, 1].flatten()
# testbody=test[:,1]
testIndex=test[:,0]



def check(word):
    word= word.lower()
    if bool(re.search(r'\d', word)) or bool(re.match(r'[^\w]', word)):
        return False
    else:
        return True

wnl = WordNetLemmatizer()
lemmatized = [None]*body.shape[0]
test_lemmatized = [None]*testbody.shape[0]
i = 0
for s in testbody:
    test_lemmatized[i] = " ".join([wnl.lemmatize(word) for word in word_tokenize(s) if (check(word))])
    i+=1
test_lemmatized=np.array(test_lemmatized)
i=0
for s in body:
    lemmatized[i] = " ".join([wnl.lemmatize(word) for word in word_tokenize(s) if (check(word))])
    i+=1
lemmatized=np.array(lemmatized)




text_classifer=Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 2),stop_words={'english'},max_df=0.9,sublinear_tf=True)),
                      ('feature_selection', SelectPercentile(f_classif, percentile=20)),
                      ('clf', SGDClassifier())])


text_classifer.fit(lemmatized,subReddit)
predict_array = text_classifer.predict(test_lemmatized)
predict_array = np.reshape(predict_array, (len(predict_array), 1))
pd.DataFrame(predict_array).to_csv("result.csv")


